In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from langchain_core.messages import HumanMessage, AIMessage

from langgraph.graph import END, StateGraph

from typing import TypedDict, List, Annotated
import operator

In [2]:
# Load API key from .env
load_dotenv()

# Set up Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-02-15-preview",
    azure_endpoint="https://exquitech-openai-2.openai.azure.com/"
)

In [3]:
class Message(TypedDict):
    role: str
    content: str

class State(TypedDict):
    messages: List[Message]

In [4]:
# State type: a simple dict that stores messages
def chat_node(state):
    messages = state["messages"]
    print("messages: ", messages)
    
    # Convert messages to OpenAI format
    chat_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
    )
    
    reply = chat_response.choices[0].message.content
    print("reply: ", reply)
    
    # Add reply to messages and return updated state
    return {"messages": messages + [{"role": "assistant", "content": reply}]}



In [ ]:
# Define LangGraph structure with state schema
builder = StateGraph(State)             # Creates a new StateGraph with the defined State schema.

builder.add_node("chat", chat_node)     # Adds a node named "chat" that runs the chat_node function.
builder.set_entry_point("chat")         # Sets the "chat" node as the entry point (where execution starts).
builder.add_edge("chat", END)           # Adds an edge from "chat" to END, indicating the graph ends after the chat node finishes.


In [ ]:
# Converts the graph structure into a runnable object.
graph = builder.compile()               # This step prepares the graph to be invoked with an initial state.

In [ ]:
# Run the graph
initial_state = {
    "messages": [{"role": "user", "content": "What is the capital of France?"}]
}

final_state = graph.invoke(initial_state)

messages:  [{'role': 'user', 'content': 'What is the capital of France?'}]
reply:  The capital of France is Paris.


- Starts the graph with an initial chat message from the user.

- Invokes the graph, which runs the chat_node once (the only node).

- The state flows through the node, OpenAI is queried, and a new assistant message is appended.

- The graph ends and returns the final state.

In [8]:
# Print the result
print(final_state["messages"][-1]["content"])

The capital of France is Paris.


- Prints the last message content, which is the assistant's reply to the question.

- For the example, you’d expect "Paris" or a similar answer from GPT-4o-mini.

In [14]:
print(final_state)
print("\n")
print(final_state["messages"])
print("\n")
print(final_state["messages"][-1])
print("\n")
print(final_state["messages"][-1]["content"])
print("\n")

{'messages': [{'role': 'user', 'content': 'What is the capital of France?'}, {'role': 'assistant', 'content': 'The capital of France is Paris.'}]}


[{'role': 'user', 'content': 'What is the capital of France?'}, {'role': 'assistant', 'content': 'The capital of France is Paris.'}]


{'role': 'assistant', 'content': 'The capital of France is Paris.'}


The capital of France is Paris.


